# 0.1 Import

In [ ]:
import pickle
import pandas as pd
import numpy as np

# 1. CLASSE Predição

In [2]:
class HealthInsurance:
    def __init__(self):
        self.home_path =                    "E:/3_recursos/2_area/profissional/cursos/22_06.1 - PA004/notebooks/10.deploy-modelo-producao.ipynb"
        self.premio_anual_mms =             "../src/features/mms_pa.pkl"
        self.idade =                        "../src/features/mms_idade.pkl"
        self.clientes_dia_contrato_mms =    "../src/features/mms_cdc.pkl"
        self.semanas_contrato_mms =         "../src/features/mms_sc.pkl"
        self.meses_contrato_mms =           "../src/features/mms_mc.pkl"
        self.genero_te =                    "../src/features/target_encode_genero.pkl"
        self.codigo_regisao_te =            "../src/features/target_encode_codigo_regiao.pkl"
        self.contato_cliente_fe =           "../src/features/fe_contato_cliente.pkl"
        
    def data_cleaning( df ):
        new_columns = ['id', 'genero', 'idade', 'codigo_regiao', 'contato_cliente', 'cnh', 'idade_veiculo',
                       'veiculo_danificado', 'seguro_previo_automovel', 'premio_anual', 'cliente_dias_contrato']
        df.columns = new_columns
        
        return df
    
    #====================
    # FEATURE ENGINNIRING
    #====================
    
    def _fe_idade_veiculo(df):
        """Codifica idade_veiculo em faixas discretas"""
        df['idade_veiculo'] = df['idade_veiculo'].apply(
            lambda x: 1 if x == '< 1 Year' else
                    2 if x == '1-2 Year' else
                    3
        )
        return df

    def _fe_veiculo_danificado(df):
        """Transforma variável binária de texto ('Yes'/'No') em 1/0"""
        df['veiculo_danificado'] = df['veiculo_danificado'].apply(
            lambda x: 1 if x == 'Yes' else 0
        )
        return df
    
    def _fe_idade_class_etaria(df):
        """Cria faixa etária com base em idade"""
        df['idade_class_etaria'] = df['idade'].apply(
            lambda x: 1 if x <= 35 else
                    2 if (x > 35) and (x <= 65) else
                    3
        )
        return df
    
    def _fe_premio_anual(df):
        """Cria categoria de prêmio anual com base em faixas de valores"""
        bins = [0, 25000, 50000, 75000, 100000, float('inf')]
        labels = [1, 2, 3, 4, 5]
        df['premio_anual_cat'] = pd.cut(
            x=df['premio_anual'],
            bins=bins,
            labels=labels,
            right=False
        ).astype('int64')
        return df

    def _fe_semanas_contrato(df):
        df['semanas_contrato'] = df['cliente_dias_contrato']/7
        return df

    def _fe_meses_contrato(df):
        df['meses_contrato'] = df['cliente_dias_contrato']/30
        return df

    def _fe_contrato(df):
        df = _fe_semanas_contrato(df)
        df = _fe_meses_contrato(df)
        return df

    DICTFE = {
        'idade_veiculo': _fe_idade_veiculo,
        'veiculo_danificado': _fe_veiculo_danificado,
        'idade': _fe_idade_class_etaria,
        'premio_anual': _fe_premio_anual,
        'cliente_dias_contrato': _fe_contrato,
    }

    def fe(df):
        for col, fn in DICTFE.items():
            if col in df.columns:
                df = fn(df)
            else:
                print(f"[Aviso] Coluna {col} ausente, pulando.")
        return df
    
    #=====================
    # NORMALIZAÇÃO
    #=====================
    
    def _norm_premio_anual(df):
        df[['premio_anual']] = np.log1p(df[['premio_anual']])
        return df

    DICTNORMALIZE = {
        'premio_anual': _norm_premio_anual
    }

    def _normalizacao(df):
        for col, fn in DICTNORMALIZE.items():
            if col in df.columns:
                df = fn(df)
            else:
                print(f"[Aviso] Coluna {col} ausente, pulando.")
        return df

    #============================
    # REESCALA
    #============================

    def _reescala_premio_anual(df):
        mms_pa = pickle.load(open('../src/features/mms_pa.pkl','rb'))
        df[['premio_anual']] = mms_pa.transform( df[['premio_anual']].values )
        return df

    def _reescala_idade(df):
        mms_idade = pickle.load(open('../src/features/mms_idade.pkl', 'rb'))
        df[['idade']] = mms_idade.transform(df[['idade']].values)
        return df

    def _reescala_cdc(df):
        mms_cdc = pickle.load(open('../src/features/mms_cdc.pkl', 'rb'))
        df[['cliente_dias_contrato']] = mms_cdc.transform(df[['cliente_dias_contrato']].values)
        return df

    def _reescala_semana_contrato(df):
        mms_sc = pickle.load(open('../src/features/mms_sc.pkl','rb' ) )
        df[['semanas_contrato']] = mms_sc.transform( df[['semanas_contrato']].values )
        return df

    def _reescala_meses_contrato(df):
        mms_mc = pickle.load(open('../src/features/mms_mc.pkl','rb' ) )
        df[['meses_contrato']] = mms_mc.transform( df[['meses_contrato']].values )
        return df

    DICTRESCALE = {
        'premio_anual': _reescala_premio_anual,
        'idade': _reescala_idade,
        'cliente_dias_contrato': _reescala_cdc,
        'semanas_contrato': _reescala_semana_contrato,
        'meses_contrato': _reescala_meses_contrato
    }

    def _reescala(df):
        for col, fn in DICTRESCALE.items():
            if col in df.columns:
                df = fn(df)
            else:
                print(f"[Aviso] Coluna {col} ausente, pulando.")
        return df

    #============================
    # ENCONDER
    #============================

    def _enc_genero(df):
        target_encode_genero = pickle.load( open ('../src/features/target_encode_genero.pkl', 'rb' ) )
        df.loc[:, 'genero'] = df['genero'].map(target_encode_genero)
        return df

    def _enc_codigo_regiao(df):
        target_encode_codigo_regiao = pickle.load( open ('../src/features/target_encode_codigo_regiao.pkl', 'rb' ) )
        df.loc[:, 'codigo_regiao'] = df['codigo_regiao'].map(target_encode_codigo_regiao)
        return df

    def _enc_idade_veiculo(df):
        df = pd.get_dummies(df, prefix=['idade_veiculo'], columns=['idade_veiculo'])
        return df

    def _enc_contato_cliente(df):
        fe_contato_cliente = pickle.load( open ('../src/features/fe_contato_cliente.pkl', 'rb' ) )
        df.loc[:, 'contato_cliente'] = df['contato_cliente'].map(fe_contato_cliente)
        return df

    DICTENCODER = {
        'genero': _enc_genero,
        'codigo_regiao': _enc_codigo_regiao,
        'idade_veiculo': _enc_idade_veiculo,
        'contato_cliente': _enc_contato_cliente
    }

    def _encoder(df):
        for col, fn in DICTENCODER.items():
            if col in df.columns:
                df = fn(df)
            else:
                print(f"[Aviso] Coluna {col} ausente, pulando.")
        return df

    #============================
    # DATA_PREPARATION
    #============================

    def data_prep(df):
        df = _normalizacao(df)
        df = _reescala(df)
        df = _encoder(df)
        
        cols_selected = ['premio_anual',
                    'idade',
                    'cliente_dias_contrato',
                    'semanas_contrato',
                    'meses_contrato',
                    'codigo_regiao',
                    'veiculo_danificado',
                    'contato_cliente',
                    'seguro_previo_automovel']
        
        df = df[cols_selected]
        
        return df
    
    def get_prediction(self, model, original_data, test_data):
        pred = model.predict_proba(test_data)
        
        original_data['prediction'] = pred
        
        return original_data.to_json( orient = "record", date_format="iso")

## API

In [ ]:
import pickle
import pandas as pd
from flask import Flask, request, Response
from deploy_test import HealthInsurance

# loading model
# path = '/Users/meigarom.lopes/repos/pa004_health_insurance_cross_sell/health_insurance_cross-sell/'
model = pickle.load( open('../src/models/lr_model.pkl', 'rb' ) )

# initialize API
app = Flask( __name__ )

@app.route( '/predict', methods=['POST'] )
def health_insurance_predict():
    test_json = request.get_json()

    if test_json: # there is data
        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )
            
        else: # multiple example
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )
            
        # Instantiate  class
        pipeline = HealthInsurance()
        
        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )
        
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )
        
        # data preparation
        df3 = pipeline.data_preparation( df2 )
        
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df3 )
        
        return df_response
        
    else:
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run( '0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.8:5000
Press CTRL+C to quit
[2026-01-15 21:20:57,404] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "e:\3_recursos\2_area\profissional\cursos\22_06.1 - PA004\.venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\3_recursos\2_area\profissional\cursos\22_06.1 - PA004\.venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\3_recursos\2_area\profissional\cursos\22_06.1 - PA004\.venv\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\3_recursos\2_area\profissional\cursos\22_06.1 - PA004\.venv\Lib\site-packages\flask\app.py", line 902, in dispatch_

## API TEST

In [ ]:
import requests
import json
import pickle as pkl

In [ ]:
df_test = pkl.load(open("../data/processed/x_validacao.pkl", 'rb'))
df_test = df_test.sample(10)
df = json.dumps(df_test.to_dict(orient = 'records'))

In [ ]:
url = "http://192.168.0.8:5000/predict"
header = {'Content-type': 'application/json'}

r = requests.post(url, data = df, headers = header)

ConnectionError: HTTPConnectionPool(host='192.168.0.8', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000215FB7471D0>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))